## EDA LAB

The General Social Survey (GSS) is a bi-annual nationally representative survey of Americans, with almost 7000 different questions asked since the survey began in the 1970s. It has straightforward questions about respondents' demographic information, but also questions like "Does your job regularly require you to perform repetitive or forceful hand movements or involve awkward postures?" or "How often do the demands of your job interfere with your family life?" There are a variety of controversial questions. No matter what you're curious about, there's something interesting in here to check out. The codebook is 904 pages (use CTRL+F to search it).

The data and codebook are available at:
https://gss.norc.org/us/en/gss/get-the-data.html

The datasets are so large that it might make sense to pick the variables you want, and then download just those variables from:
https://gssdataexplorer.norc.org/variables/vfilter

Here is your task:
1. Download a small (5-15) set of variables of interest.
2. Write a short description of the data you chose, and why. (1 page)
3. Load the data using Pandas. Clean them up for EDA. Do this in a notebook with comments or markdown chunks explaining your choices.
4. Produce some numeric summaries and visualizations. (1-3 pages)
5. Describe your findings in 1-2 pages.
6. If you have other content that you think absolutely must be included, you can include it in an appendix of any length.

For example, you might want to look at how aspects of a person's childhood family are correlated or not with their career or family choices as an adult. Or how political or religious affiliations correlate with drug use or sexual practices. It's an extremely wide-ranging survey.

Feel free to work with other people in groups, and ask questions!

In [2]:
# 1) Loading Variables --

import pandas as pd
var_list = ['prestg10', 'educ', 'race', 'mawrkgrw', 'unrelat', 'colath', 'popular', 'divlaw', 'letdie1', 'helpfulv'] # List of variables you want to save
output_filename = 'selected_gss_data.csv' # Name of the file you want to save the data to
phase = 0 # Starts in write mode; after one iteration of loop, switches to append mode
for k in range(3): # for each chunk of the data
    url = 'https://github.com/DS3001/project_gss/raw/main/gss_chunk_' + str(1+k) + '.parquet' # Create url to the chunk to be processed
    print(url) # Check the url is correct
    df = pd.read_parquet(url) # Download this chunk of data
    print(df.head()) # Visually inspect the first few rows
    if phase == 0 :
        df.loc[:,var_list].to_csv(output_filename, # specifies target file to save the chunk to
                                mode='w', # control write versus append
                                header=var_list, # variable names
                                index=False) # no row index saved
        phase = 1 # Switch from write mode to append mode
    elif phase == 1 :
        df.loc[:,var_list].to_csv(output_filename, # specifies target file to save the chunk to
                                mode='a', # control write versus append
                                header=None, # variable names
                                index=False) # no row index saved
        phase = 1 # Switch from write mode to append mode

https://github.com/DS3001/project_gss/raw/main/gss_chunk_1.parquet
   year  id            wrkstat  hrs1  hrs2 evwork    occ  prestige  \
0  1972   1  working full time   NaN   NaN    NaN  205.0      50.0   
1  1972   2            retired   NaN   NaN    yes  441.0      45.0   
2  1972   3  working part time   NaN   NaN    NaN  270.0      44.0   
3  1972   4  working full time   NaN   NaN    NaN    1.0      57.0   
4  1972   5      keeping house   NaN   NaN    yes  385.0      40.0   

         wrkslf wrkgovt  ...  agehef12 agehef13 agehef14  hompoph wtssps_nea  \
0  someone else     NaN  ...       NaN      NaN      NaN      NaN        NaN   
1  someone else     NaN  ...       NaN      NaN      NaN      NaN        NaN   
2  someone else     NaN  ...       NaN      NaN      NaN      NaN        NaN   
3  someone else     NaN  ...       NaN      NaN      NaN      NaN        NaN   
4  someone else     NaN  ...       NaN      NaN      NaN      NaN        NaN   

   wtssnrps_nea  wtssps_next wt

**2) Short Description of the data you chose and why:**

Many of the variables we picked were from controversial questions that attacked or related to specific American values ranging from work ethic, family values, community, work competition, and medical decisions. This data goes beyond the staple demographic data and reveals more about both the ideology and environments of a number of Americans that build up their beliefs and perspectives. We tried to pick a range of variables that touched on various aspects of life including the relationships indivdiuals have with their community, their home life, experinces at work, and their own children. One of the variables that particularly stood out was a question about having a working mother during an individual's teens. This variable was particularly interesting to me as it revealed a bias in the questions on the survey. Some of the variables, the more controversial ones, were chosen for this similar reason.

In [6]:
# 3) Load the Data and do clean up
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# load the dataset
gss_df = pd.read_csv("selected_gss_data.csv")

# display basic information -- to help analyze what needs to be cleaned
print(gss_df.info())
print(gss_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72390 entries, 0 to 72389
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   prestg10  67248 non-null  float64
 1   educ      72127 non-null  float64
 2   race      72283 non-null  object 
 3   mawrkgrw  39141 non-null  object 
 4   unrelat   49369 non-null  float64
 5   colath    44800 non-null  object 
 6   popular   31807 non-null  object 
 7   divlaw    38623 non-null  object 
 8   letdie1   35537 non-null  object 
 9   helpfulv  1927 non-null   object 
dtypes: float64(3), object(7)
memory usage: 5.5+ MB
None
   prestg10  educ   race mawrkgrw  unrelat                 colath popular  \
0      45.0  16.0  white      NaN      NaN  yes, allowed to teach     NaN   
1      50.0  10.0  white      NaN      0.0            not allowed     NaN   
2      49.0  12.0  white      NaN      0.0            not allowed     NaN   
3      60.0  17.0  white      NaN      0.0            not 

In [7]:
# Cleaning up data: first step - handling significant missing values

# Column has over 97% missing values, not helpful in doing any sort of analysis for this column
gss_df.drop(columns=["helpfulv"], inplace=True)
# Getting rid of columns that are missing and replace missing values with median values
gss_df.dropna(subset=["prestg10", "educ", "race"], inplace=True)
gss_df["prestg10"].fillna(df["prestg10"].median(), inplace=True)
gss_df["unrelat"].fillna(df["unrelat"].median(), inplace=True)
# Replace categorical variables with Unknown for missing values
gss_df.fillna({"mawrkgrw": "Unknown", "colath": "Unknown", "popular": "Unknown",
           "divlaw": "Unknown", "letdie1": "Unknown"}, inplace=True)
# Save the cleaned data into a sperate csv file
gss_df.to_csv("gss_cleaned.csv", index=False)

<ipython-input-7-29807f67f71b>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  gss_df["prestg10"].fillna(df["prestg10"].median(), inplace=True)
<ipython-input-7-29807f67f71b>:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=Tr